In [ ]:
%matplotlib notebook

import os
import datetime as dt
import pickle, joblib


# Standard data science libraries
import pandas as pd
import numpy as np
import scipy.stats as ss
import scipy.optimize as so 
import scipy.interpolate as si

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-notebook')

# Options for pandas
pd.options.display.max_columns = 20
pd.options.display.max_rows = 200

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython.display import Image
from IPython.display import Math



In [ ]:
from ipywidgets import interact, Dropdown
from IPython.display import display

import flopy as fp
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point

import RTD_util6 as rtd_ut
import Genmod_Utilities as gmu

import matplotlib.dates as mdates
import matplotlib.ticker as mticks

import json

The next cell sets up some color and font choices that work for AGU journals.

In [ ]:
KS1 = '#06366E'
KS2 = '#00A3EB'
KS3 = '#25C0A6'
KS4 = '#FDDA58'
KS5 = '#5D171A'

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 12,
        'sans-serif' : 'Arial'}

plt.rc('font', **font)

The user can set the number of particles, porosity, and location of the MODPATH7 executable file. The number of particles that can be used is proportional to the amount of RAM availble. $4*10^6$ works with 64 GB of RAM, although more may be possible. The constant value for porosity can be replaced with a numpy array.

In [ ]:
total_number_of_particles = 4.E+06
por = 0.20
mp_exe_name7 = '../Executables/modpath_7_2_001/bin/mpath7.exe'

Read in some files created by previous notebooks.

In [ ]:
with open('GenMod_metadata.txt') as json_file:
    metadata = json.load(json_file)
    
src = os.path.join('model_ws', 'gsm_metadata.json')
with open(src, 'r') as f:
    gsm_metadata = json.load(f)   
    
from argparse import Namespace
meta = Namespace(**gsm_metadata)

## Use General Simulation Model to calculate TTD

Read MODFLOW model and create RTD object

In [ ]:
print('Reading model information')

ml = fp.mf6.MFSimulation.load(sim_name='mfsim.nam', version='mf6', exe_name=metadata['modflow_path'],
                              sim_ws='optimal_model', strict=True, verbosity_level=0, load_only=None, verify_data=False)
model = ml.get_model()
rtd = rtd_ut.RTD_util(ml, 'flow', 'rt')
print('   ... done')

## Read model output and compute net inflow to drain cells

In [ ]:
# read shapefile created in step 1--NHD flowlines intersected with model grid
src = os.path.join('gis', 'drain_segments.shp')
shp = gpd.read_file(src)

# read enhanced model_grid file in model_ws
src = os.path.join('gis', 'model_grid.csv')
data = pd.read_csv(src)

# extract the drain budget terms from modflow output
rtd.get_budget('DRN')
drains = rtd.budget

# create a dataframe of drain flows
drn_df = pd.DataFrame(drains[0])
drn_df['node'] = drn_df['node'] - 1

# merge drain segments (by model cells) with drain flows
shp_drn_df = shp.merge(drn_df, left_on='node', right_on='node', how='outer')
shp_drn_df = shp_drn_df[shp_drn_df.q < 0]

# save shapefile to model_ws
dst = os.path.join('optimal_model', 'drain_flows.shp')
shp_drn_df.to_file(dst)

flow = drn_df.q.sum()

particles_per_flow = total_number_of_particles / flow

# make particle locations
x_partloc, y_partloc, node_list, label_list = rtd.make_stream_particle_array(
    shp_drn_df, data, particles_per_flow, seg_ref='NHDPlusID')

label_list = [np.int32(str(item)[-9:]) for item in label_list]

Test case for local coordinates for stream particles

In [ ]:
# rtd.run_test(100)

* Write starting particle location file.
* Particles start on top face of drain cells
* Number of particles is proportional to new flux through top face
* Label particles with the NHD reachcode

In [ ]:
particle_data = fp.modpath.ParticleData(partlocs=node_list,
                                        structured=False,
                                        particleids=label_list,
                                        localx=x_partloc,
                                        localy=y_partloc,
                                        localz=1,
                                        timeoffset=None,
                                        drape=0)

particle_group = fp.modpath.ParticleGroup(particlegroupname='drains', 
                                          filename='particles.loc',
                                          releasedata=0,
                                          particledata=particle_data)

Run MODPATH

In [ ]:
mpname = '{}_{}_{}'.format(model.name, 'volume', 'rt')
mpnf = '{}_{}_{}.mpnam'.format(model.name, 'volume', 'rt')
mplf = '{}_{}_{}.mplst'.format(model.name, 'volume', 'rt')
headfile = model.oc.head_filerecord.array.getfield('O')[0]
budfile = model.oc.budget_filerecord.array.getfield('O')[0]
endpointfile = '{}.mpend'.format(model.name)

mp = fp.modpath.Modpath7(modelname=mpname, 
                         simfile_ext='mpsim', 
                         namefile_ext='mpnam', 
                         version='modpath7',
                         exe_name=mp_exe_name7, 
                         flowmodel=model, 
                         headfilename=headfile,
                         budgetfilename=budfile,
                         model_ws='optimal_model',
                         verbose=True)

mpsim = fp.modpath.Modpath7Sim(mp, 
                               mpnamefilename=mpnf,
                               listingfilename=mplf,
                               endpointfilename=endpointfile,
                               simulationtype='endpoint',
                               trackingdirection='backward',
                               weaksinkoption='stop_at',
                               weaksourceoption='stop_at',
                               budgetoutputoption='summary',
                               referencetime=rtd.ref_time,
                               stoptimeoption='extend',
                               zonedataoption='off',
                               stopzone=-1,
                               particlegroups=particle_group,
                               extension='mpsim')

mpbas = fp.modpath.Modpath7Bas(mp,
                               porosity=0.20,
                               defaultiface={'DRN': 6, 'RCH': 6})

mp.write_input()
success, msg = mp.run_model(silent=True, report=False)

Read endpoint information

In [ ]:
ep_data = rtd.read_endpoints(os.path.join('optimal_model', endpointfile))

Write modified endpoint file

In [ ]:
rtd.modify_endpoint_file(ep_data, write=True)
try:
    os.remove(os.path.join('optimal_model', endpointfile))
except FileNotFoundError:
    pass
try:
    os.remove(os.path.join('optimal_model', 'particles.loc'))
except FileNotFoundError:
    pass

